In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# load data

In [ ]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Flooded SNMP packets detection/transormed_SNMPdata_binary.csv')
#df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Flooded SNMP packets detection/transormed_SNMPdata_multiclass.csv')

In [ ]:
df = df.iloc[:,2:]
df = df.reset_index(drop=True)
# df.head()

In [ ]:
print(df.shape)
df.head()

(397, 174)


,1,2,3,4,5,6,7,8,9,10,...,165,166,167,168,169,170,171,172,173,174
0,16,0,17,1,0,0,17,1,16,0,...,0,273,0,296,2,0,296,2,296,0
1,108,0,108,1,0,0,91,1,91,0,...,0,273,0,296,2,0,296,2,296,0
2,98,0,98,1,0,0,82,1,97,0,...,0,299,0,316,2,0,296,2,296,0
3,93,0,93,1,0,0,93,1,108,0,...,0,299,0,316,2,0,296,2,296,0
4,104,0,104,1,0,0,2618,1,2617,0,...,0,299,0,316,2,0,296,2,296,0


In [ ]:
#check whether there are any null value
df.isnull().values.any()

define X and Y from dataset to rain and test

In [ ]:
X = df.iloc[:,:-1]
print(X.shape)
Y = df.iloc[:,-1]
sc_ = StandardScaler()
sc_.fit(X)
X_std = sc_.transform(X)
print(X_std.shape)

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X, Y, test_size=0.3,random_state=0)
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)
X_combined_std = np.vstack((X_train_std,X_test_std))
y_combined = np.hstack((y_train,y_test))

In [ ]:
def calculate_Fscore(conf_matrix):# order : 0 tn, 1 fp, 2 fn, 3 tp
    accuracy=(conf_matrix[3]+conf_matrix[0])/(conf_matrix[3]+conf_matrix[2]+conf_matrix[0]+conf_matrix[1])*100
    precision=conf_matrix[3]/(conf_matrix[3]+conf_matrix[1])
    recall=conf_matrix[3]/(conf_matrix[3]+conf_matrix[2])
    speficity=conf_matrix[0]/(conf_matrix[0]+conf_matrix[1])
    fscore=2*(precision*recall)/(precision+recall)
    NegPredictiveVal=conf_matrix[0]/(conf_matrix[0]+conf_matrix[2])
    print('conf_matrix : ',conf_matrix)
    print('accuracy : ',accuracy)
    print('fscore : ',fscore)
    print('precision : ',precision)
    print('recall : ',recall)
    print('NegPredictiveVal : ',NegPredictiveVal)
    print('speficity : ',speficity)

    #print(log_loss)
    return([accuracy,fscore])

# Algorithms Testing

In [ ]:
from sklearn.svm import SVC
svc = SVC(kernel='linear', C=1.0, random_state=0)
svc.fit(X_train_std, y_train)
# print("Accuracy on training set: {:.3f}".format(svm.score(X_train_std,y_train)))
# print("Accuracy on test set: {:.3f}".format(svm.score(X_test_std,y_test)))
print(cross_val_score(svc, X_std, Y, cv=3))
print("The average cross validation score is: {:.3f}".format(np.mean(cross_val_score(svc, X_std, Y, cv=3))))
#LINEAR(71) > SIGMOID(63) > POLY(62) > RBF(61)

[0.61654135 0.68939394 0.68939394]
The average cross validation score is: 0.665


In [ ]:
from sklearn.svm import NuSVC
nusvc = NuSVC()
nusvc.fit(X_train_std, y_train)
# print("Accuracy on training set: {:.3f}".format(svm.score(X_train_std,y_train)))
# print("Accuracy on test set: {:.3f}".format(svm.score(X_test_std,y_test)))
print(cross_val_score(nusvc, X_std, Y, cv=3))
print("The average cross validation score is: {:.3f}".format(np.mean(cross_val_score(nusvc, X_std, Y, cv=3))))
#LINEAR(71) > SIGMOID(63) > POLY(62) > RBF(61)

[0.72592593 0.94776119 0.89552239]
The average cross validation score is: 0.856


In [ ]:
from sklearn.svm import LinearSVC
lin_svc = LinearSVC()
lin_svc.fit(X_train_std, y_train)
# print("Accuracy on training set: {:.3f}".format(svm.score(X_train_std,y_train)))
# print("Accuracy on test set: {:.3f}".format(svm.score(X_test_std,y_test)))
print(cross_val_score(lin_svc, X_std, Y, cv=3))
print("The average cross validation score is: {:.3f}".format(np.mean(cross_val_score(lin_svc, X_std, Y, cv=3))))
#LINEAR(71) > SIGMOID(63) > POLY(62) > RBF(61)

[0.73333333 0.96268657 0.68656716]
The average cross validation score is: 0.794


In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train_std, y_train)

print(cross_val_score(gnb, X_std, Y, cv=3))
print("The average cross validation score is: {:.3f}".format(np.mean(cross_val_score(gnb, X_std, Y, cv=3))))

[0.72592593 0.64179104 0.68656716]
The average cross validation score is: 0.685


In [ ]:
from sklearn.naive_bayes import BernoulliNB
bnb = BernoulliNB()
bnb.fit(X_train_std, y_train)

print(cross_val_score(bnb, X_std, Y, cv=3))
print("The average cross validation score is: {:.3f}".format(np.mean(cross_val_score(bnb, X_std, Y, cv=3))))

[0.73333333 0.82835821 0.82089552]
The average cross validation score is: 0.794


In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(C=1000.0, random_state=0)
lr.fit(X_train_std,y_train)
# print("Accuracy on training set: {:.3f}".format(lr.score(X_train_std,y_train)))
# print("Accuracy on test set: {:.3f}".format(lr.score(X_test_std,y_test)))
print(cross_val_score(lr, X_std, Y, cv=3))
print("The average cross validation score is: {:.3f}".format(np.mean(cross_val_score(lr, X_std, Y, cv=3))))

[0.6        0.94776119 0.68656716]
The average cross validation score is: 0.745


In [ ]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(random_state=0)
tree = DecisionTreeClassifier(criterion='entropy',max_depth = 4,random_state=0)
tree.fit(X_train_std,y_train)
# print("Accuracy on training set: {:.3f}".format(tree.score(X_train_std,y_train)))
# print("Accuracy on test set: {:.3f}".format(tree.score(X_test_std,y_test)))
print(cross_val_score(tree, X_std, Y, cv=3))
print("The average cross validation score is: {:.3f}".format(np.mean(cross_val_score(tree, X_std, Y, cv=3))))

[0.77037037 0.99253731 0.8880597 ]
The average cross validation score is: 0.884


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(X_train_std,y_train)
# print("Accuracy on training set: {:.3f}".format(neigh.score(X_train_std,y_train)))
# print("Accuracy on test set: {:.3f}".format(neigh.score(X_test_std,y_test)))
print(cross_val_score(neigh, X_std, Y, cv=3))
print("The average cross validation score is: {:.3f}".format(np.mean(cross_val_score(neigh, X_std, Y, cv=3))))

[0.64661654 0.78787879 0.75      ]
The average cross validation score is: 0.728


In [ ]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
rf = RandomForestClassifier(n_estimators=7, max_depth=None, min_samples_split=2, random_state=0)
rf.fit(X_train_std,y_train)
# print("Accuracy on training set: {:.3f}".format(rf.score(X_train_std,y_train)))
# print("Accuracy on test set: {:.3f}".format(rf.score(X_test_std,y_test)))
print(cross_val_score(rf, X_std, Y, cv=3))
print("The average cross validation score is: {:.3f}".format(np.mean(cross_val_score(rf, X_std, Y, cv=3))))

[0.8962963  1.         0.89552239]
The average cross validation score is: 0.931


In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
xtra = ExtraTreesClassifier(n_estimators=6, max_depth=None, min_samples_split=2, random_state=0)
xtra.fit(X_train_std,y_train)
print(cross_val_score(xtra, X_std, Y, cv = 3))
print("The average cross validation score is: {:.3f}".format(np.mean(cross_val_score(xtra, X_std, Y, cv=3))))

[0.81203008 1.         0.96969697]
The average cross validation score is: 0.927


1.0

## boosting alorithms

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm.sklearn import LGBMClassifier
from xgboost import XGBClassifier

In [ ]:
xg = XGBClassifier()
xg.fit(X_train_std,y_train)
print(cross_val_score(xg, X_std, Y, cv=3))
print("The average cross validation score is: {:.3f}".format(np.mean(cross_val_score(xg, X_std, Y, cv=3))))

[0.66165414 1.         0.84848485]
The average cross validation score is: 0.837


In [ ]:
lgb = LGBMClassifier()
lgb.fit(X_train_std,y_train)
print(cross_val_score(lgb, X_std, Y, cv=3))
print("The average cross validation score is: {:.3f}".format(np.mean(cross_val_score(lgb, X_std, Y, cv=3))))

[0.78947368 1.         0.84848485]
The average cross validation score is: 0.879


In [ ]:
adb = AdaBoostClassifier()
adb.fit(X_train_std,y_train)
print(cross_val_score(adb, X_std, Y, cv=3))
print("The average cross validation score is: {:.3f}".format(np.mean(cross_val_score(adb, X_std, Y, cv=3))))

[0.29323308 0.48484848 0.58333333]
The average cross validation score is: 0.506


In [ ]:
gbm = GradientBoostingClassifier()
gbm.fit(X_train_std,y_train)
print(cross_val_score(gbm, X_std, Y, cv=3))
print("The average cross validation score is: {:.3f}".format(np.mean(cross_val_score(gbm, X_std, Y, cv=3))))

## GAM

In [ ]:
!pip install pygam
from pygam import LogisticGAM

In [ ]:
X_train_std.isnull().values.any()

In [ ]:
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_validate
from sklearn.metrics import confusion_matrix
gam = LogisticGAM()
gam.fit(X_train_std,y_train)

predictions=cross_val_predict(gam,X_std,Y,cv=4)
cm=list(confusion_matrix(Y,predictions).ravel())
calculate_Fscore(cm)

# print(cross_val_score(gam, X_std, Y, cv=3,scoring='accuracy'))
# print("The average cross validation score is: {:.3f}".format(np.mean(cross_val_score(gam, X_std, Y, cv=3))))

# ensemble classificattion testing

svc
/
lin_svc
/
nusvc
/
gnb
/
bnb
/
lr
/
tree
/
neigh
/
rf
/
xtra
/
adb
/
gbm
/
xg
/
lgb




In [ ]:
estimators = [
              # ('svc', svc ), #0.665
              # ('lin_svc', lin_svc ), #0.733
              # ('nusvc', nusvc ), #0.680
              ('gnb', gnb ), #0.748
              # ('bnb', bnb ), #0.653
              #('lr',  lr), #0.711
              # ('tree', tree), #0.877
              # ('neigh', neigh ), #0.728
              ('rf', rf ), #0.910
              ('xtra', xtra ), # 0.927
              ('gbm', gbm), #0.889
              ('xg', xg ), #0.837
              # ('lgb', lgb ), #0.879
              ]

In [ ]:
from sklearn.ensemble import StackingClassifier
clf = StackingClassifier(estimators=estimators, cv=4, final_estimator=LogisticRegression())
clf.fit(X_train_std,y_train)
print("The average cross validation score is: {:.3f}".format(np.mean(cross_val_score(clf, X_std, Y, cv=3))))

The average cross validation score is: 0.687


### automated ensembling

In [ ]:
import random

baseLeaners = [ svc , lin_svc , nusvc , gnb , bnb , lr , tree , neigh , rf  , adb , gbm , xg ,lgb ] #, xtra
tmp=baseLeaners
estimator_name=0
estimators =[(str(estimator_name),xtra)]
acc=0.931;acc_new=0.0

while acc_new<acc:
  num = random.randint(0, 12)
  estimator_name+=1
  print(estimator_name)
  estimators.append((str(estimator_name),baseLeaners[num]))
  # print(num,estimators)

  clf = StackingClassifier(estimators=estimators, cv=4, final_estimator=LogisticRegression())
  clf.fit(X_train_std,y_train)
  acc_new=np.mean(cross_val_score(clf, X_std, Y, cv=3))

  if acc_new>acc:
    tmp.remove(baseLeaners[num])
    print(acc_new)

print(estimators)

In [ ]:
print(estimators)

# Save models

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
xtra2 = ExtraTreesClassifier(n_estimators=6, max_depth=None, min_samples_split=2, random_state=0)
xtra2.fit( X_std, Y)
print(cross_val_score(xtra2, X_std, Y, cv = 3))
print("The average cross validation score is: {:.3f}".format(np.mean(cross_val_score(xtra2, X_std, Y, cv=3))))

In [ ]:
from sklearn.externals import joblib

#dont run
#joblib.dump(xtra2, 'pickle_model_xtra.pkl') #multiclass
# joblib.dump(rf2, 'pickle_model_rf.pkl') #binary

['pickle_model_xtra.pkl']

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# get the folder id where you want to save your file
file = drive.CreateFile({'parents':[{u'id': '1IRDp6iQfoGfK0oXrH7BOjWqyUaq5tHvl'}]})
file.SetContentFile(pkl_filename)
# file.Upload() #dont run